***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-8605  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 28 10:09 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'bucket_wiki1' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs_title = parquetFile.select("title", "id").rdd
doc_text_pairs_anchor = parquetFile.select("anchor_text", "id").rdd
doc_text_pairs_body = parquetFile.select("text", "id").rdd


In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import InvertedIndex

In [12]:
import math
import statistics
bucket_name = 'ir_resources'
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
    
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    map = {}
    for w in tokens:
        if w in all_stopwords:
            continue
        map[w] = map.get(w, 0) + 1
    return [(k,(id,v)) for k,v in map.items()]

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

def calculate_df(postings):
    return postings.map(lambda x:(x[0],len(x[1])))

def partition_postings_and_write(postings,base_dir):   
    posting_lst_with_bucket = postings.map(lambda x:((token2bucket_id(x[0]),x)))
    posting_lst_group_by_bucket = posting_lst_with_bucket.groupByKey()
    return posting_lst_group_by_bucket.map(lambda x :InvertedIndex.write_a_posting_list(x,bucket_name,base_dir))
def calculate_dl(postings):
    return postings.map(lambda x:x[1]).flatMap(lambda x:x).reduceByKey(lambda x,y :x+y)
def calculate_tt(postings):
    return postings.map(lambda x: (x[0], np.sum([y[1] for y in x[1]])))
def calculate_nf(postings):
    return postings.map(lambda x: x[1]).flatMap(lambda x:x).map(lambda x:(x[0],x[1]**2)).groupByKey().mapValues(list).map(lambda x:(x[0],1/math.sqrt(np.sum(x[1]))))
def calculate_AVGDL(DL):
    return statistics.mean(DL.values())


# title inverted index

In [13]:
word_counts_title = doc_text_pairs_title.flatMap(lambda x: word_count(x[0], x[1]))

In [14]:
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [15]:
w2df_title = calculate_df(postings_title).collectAsMap()

In [16]:
tt_title = calculate_tt(postings_title).collectAsMap()

In [17]:
DL_title = calculate_dl(postings_title).collectAsMap()

In [18]:
nf_title = calculate_nf(postings_title).collectAsMap()

In [19]:
_ = partition_postings_and_write(postings_title,"title_index").collect()

In [20]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_title = pickle.load(f)
        for k, v in posting_locs_title.items():
            super_posting_locs_title[k].extend(v)

Putting it all together

In [21]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_title
inverted_title.DL = DL_title
inverted_title.term_total = tt_title
inverted_title.nf = nf_title
inverted_title.AVGDL = calculate_AVGDL(DL_title)

# write the global stats out
inverted_title.write_index('.', 'title_index')

# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][256.4 MiB/256.4 MiB]                                                
Operation completed over 1 objects/256.4 MiB.                                    


In [22]:
!gsutil ls -lh $index_dst

256.38 MiB  2022-12-28T15:24:29Z  gs://ir_resources/title_index/title_index.pkl
TOTAL: 1 objects, 268831615 bytes (256.38 MiB)


# body inverted index

In [23]:
word_counts_body = doc_text_pairs_body.flatMap(lambda x: word_count(x[0], x[1]))

In [24]:
postings_body = word_counts_body.groupByKey().mapValues(reduce_word_counts)
postings_body_filtered = postings_body.filter(lambda x: len(x[1])>50)

In [ ]:
w2df_body = calculate_df(postings_body_filtered).collectAsMap()

In [ ]:
tt_body = calculate_tt(postings_body_filtered).collectAsMap()

In [ ]:
DL_body = calculate_dl(postings_body_filtered).collectAsMap()

In [ ]:
nf_body = calculate_nf(postings_body_filtered).collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings_body_filtered,"body_index").collect()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_body = pickle.load(f)
        for k, v in posting_locs_body.items():
            super_posting_locs_body[k].extend(v)

In [ ]:
# Create inverted index instance
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_body
inverted_body.DL = DL_body
inverted_body.term_total = tt_body
inverted_body.nf = nf_body
inverted_body.AVGDL = calculate_AVGDL(DL_body)

# write the global stats out
inverted_body.write_index('.', 'body_index')

# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/body_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][164.3 MiB/164.3 MiB]                                                
Operation completed over 1 objects/164.3 MiB.                                    


In [ ]:
!gsutil ls -lh $index_dst

164.29 MiB  2022-12-28T16:41:20Z  gs://ir_resources/body_index/body_index.pkl
TOTAL: 1 objects, 172271764 bytes (164.29 MiB)


# anchor inverted index

In [38]:
doc_text_pairs_anchor = parquetFile.select("id","anchor_text").rdd
postings_doc_lst_titles = doc_text_pairs_anchor.map(lambda x:[(y[0], y[1]) for y in x[1]])
postings_doc_lst_titles = postings_doc_lst_titles.flatMap(lambda x:[(w ,id) for id,text in x for w in text.split()])
postings_term_to_docs = postings_doc_lst_titles.groupByKey().mapValues(set).map(lambda x:(x[0],[(doc_id ,1)for doc_id in x[1]]))


In [ ]:
_ = partition_postings_and_write(postings_term_to_docs,"anchor_index").collect()

In [44]:
super_posting_locs_anchor = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor_index'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs_anchor = pickle.load(f)
        for k, v in posting_locs_anchor.items():
            super_posting_locs_anchor[k].extend(v)

In [45]:
# Create inverted index instance
inverted_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_anchor.posting_locs = super_posting_locs_anchor

# write the global stats out
inverted_anchor.write_index('.', 'anchor_index')

# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/anchor_index/{index_src}'

In [46]:
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
\ [1 files][ 89.0 MiB/ 89.0 MiB]                                                
Operation completed over 1 objects/89.0 MiB.                                     


# title to doc

In [48]:
docid_title_dict = parquetFile.select("id","title").rdd.collectAsMap()
with open("docid_title_dict.pkl", 'wb') as f:
    pickle.dump(docid_title_dict, f)

In [49]:
index_src = "docid_title_dict.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'

In [50]:
!gsutil cp $index_src $index_dst

Copying file://docid_title_dict.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][168.9 MiB/168.9 MiB]                                                
Operation completed over 1 objects/168.9 MiB.                                    
